# ALTEGRAD Challenge - Features generation

*Abderrahim AIT-AZZI, Sébastien OHLEYER, Mickael SUTTON*

In [4]:
# No import needed

## 1. WMD distance, Sent2vec, Glove

In [5]:
from features_engineering.glove import generate_glove_features
path='./data'

In [6]:
word2vec_filepath = ''
googlenews_filepath = ''
generate_glove_features(path, word2vec_filepath, googlenews_filepath)

RuntimeError: invalid file URI: 

# 2. Pagerank

In [1]:
from features_engineering.pagerank import generate_pagerank
path = "./data"

In [2]:
generate_pagerank(path)

Apply to train...
Apply to test...
Main PR generator...
Apply to train...
Writing train features...
Apply to test...
Writing test features...
CSV written ! see:  ./data  | suffix:  _pagerank.csv


# 3. Question frequency

In [3]:
from features_engineering.question_freq import generate_question_freq
path = "./data"

In [4]:
generate_question_freq(path)

Writing train features...
Writing test features...
CSV written ! see:  ./data  | suffix:  _question_freq.csv


# 4. Intersection of questions

In [5]:
from features_engineering.question_inter import generate_question_inter
path = "./data"

In [6]:
generate_question_inter(path)

Writing train features...
Writing test features...
CSV written ! see:  ./data  | suffix:  _question_inter.csv



# 5. K cores

In [1]:
from features_engineering.question_kcores import generate_question_kcores
path='./data'

In [2]:
generate_question_kcores(path)

  0%|          | 0/29 [00:00<?, ?it/s]/Users/sebastienohleyer/Documents/ENS MVA/altegrad/features_engineering/question_kcores.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['kcores'][df.question.isin(ck)] = k
100%|██████████| 29/29 [00:20<00:00,  1.39it/s]


Writing train features...
Writing test features...
CSV written ! see:  ./data  | suffix:  _question_kcores.csv


# 6. TF-IDF

In [1]:
from features_engineering.tfidf import generate_tfidf
path = "./data/"

In [2]:
generate_tfidf(path)

Building Features :
word_match
tfidf
tfidf_wm_stops
jaccard, wc_diff, wc_ratio, wc_diff_unique, wc_ratio_unique
wc_diff_unq_stop, wc_ratio_unique_stop
same_start, char_diff
char_diff_unq_stop
total_unique_words
total_unq_words_stop
char_ratio
word_match
tfidf
tfidf_wm_stops
jaccard, wc_diff, wc_ratio, wc_diff_unique, wc_ratio_unique
wc_diff_unq_stop, wc_ratio_unique_stop
same_start, char_diff
char_diff_unq_stop
total_unique_words
total_unq_words_stop
char_ratio
Writing train features...
Writing test features...
CSV written ! see:  ./data/  | suffix:  _tfidf.csv


# 7. Graph features

## 7.1 Unweighted graph

In [5]:
from features_engineering.graph_features import generate_graph_features
path = './data'

In [6]:
generate_graph_features(path)

2it [00:00, 19.90it/s]

Number of nodes: 58940
Number of edges: 100279
Computing train features


80100it [17:59, 74.21it/s]


Writing train features...


7it [00:00, 65.31it/s]

Computing test features


20179it [03:52, 86.86it/s]


Writing test features...
CSV written ! see:  ./data  | suffix:  _graph_feat.csv


## 7.2 Weighted graph

In [13]:
from features_engineering.weightedgraph_features import generate_weightedgraph_features
path = './data'

In [2]:
manual_cv = False
generate_weightedgraph_features(path, manual_cv)

27it [00:00, 269.64it/s]

Number of nodes: 58940
Number of edges: 100279
Computing train features


80100it [04:52, 273.66it/s]
41it [00:00, 401.22it/s]

Computing test features


20179it [01:00, 331.65it/s]


Writing train features...
Writing test features...
CSV written ! see:  ./data  | suffix:  _weightedgraph_feat.csv


# 8. N-grams

In [1]:
from features_engineering.cooccurence_distinct_ngram import generate_cooccurence_distinct_ngram
path = './data'

In [2]:
generate_cooccurence_distinct_ngram(path,3)

9it [00:00, 85.61it/s]

Applying to train...


80100it [14:20, 93.08it/s]


Writing train features...


14it [00:00, 137.09it/s]

Applying to test...


20179it [03:00, 111.68it/s]


Writing test features...
CSV written ! see:  ./data  | suffix:  _3gram_feat.csv


# 9. Word features

In [14]:
from features_engineering.word_features import generate_word_features
path='./data'

/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
generate_word_features(path)

/Users/sebastienohleyer/Documents/ENS MVA/alte/features_engineering/word_features.py:48: RuntimeWarning: invalid value encountered in long_scalars
  R1 = np.sum(shared_weights) / np.sum(total_weights) #tfidf share
/Users/sebastienohleyer/Documents/ENS MVA/alte/features_engineering/word_features.py:53: RuntimeWarning: invalid value encountered in true_divide
  Rcosine = np.dot(shared_weights, shared_weights)/Rcosine_denominator
/Users/sebastienohleyer/Documents/ENS MVA/alte/features_engineering/word_features.py:48: RuntimeWarning: invalid value encountered in double_scalars
  R1 = np.sum(shared_weights) / np.sum(total_weights) #tfidf share


Writing train features...
Writing test features...
CSV written ! see:  ./data  | suffix:  _word_feat.csv


# 10. SpaCy

In [15]:
from features_engineering.spacy_features import generate_spacy_features
path = './data'

In [25]:
generate_spacy_features(path)

4it [00:00, 32.93it/s]

Applying to train...


80100it [48:53, 27.31it/s]


Writing train features...


2it [00:00, 18.54it/s]

Applying to train...


20179it [11:35, 29.02it/s]


Writing test features...
CSV written ! see:  ./data  | suffix:  _spacy_features.csv


# 11. Letter count features

In [16]:
from features_engineering.letters_count_feat import generate_letters_count_features
path = './data'

In [11]:
generate_letters_count_features(path)

Writing train features...
CSV written ! see:  ./data  | suffix:  _count_feat.csv
Writing test features...
CSV written ! see:  ./data  | suffix:  _count_feat.csv
